In [ ]:
import os
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET
from openpyxl import load_workbook
import sqlite3
import pandas as pd

# Prompt user to select a folder starting with the specified path
root = tk.Tk()
root.withdraw()  # Hide the root window
selected_folder = filedialog.askdirectory(initialdir='D:/Readiness Screen 3/Data/')

if not selected_folder:
    print("No folder selected. Exiting...")
    exit()

# Find the XML file titled "Session" in the selected folder
xml_file_path = ''
for root_dir, dirs, files in os.walk(selected_folder):
    for file in files:
        if file.lower().startswith('session') and file.lower().endswith('.xml'):
            xml_file_path = os.path.join(root_dir, file)
            break
    if xml_file_path:
        break

if not xml_file_path:
    print("No XML file titled 'Session' found in the selected folder. Exiting...")
    exit()

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Function to safely find text of an element
def find_text(element, tag):
    found = element.find(tag)
    return found.text if found is not None else None

# Extract required fields from the 'Session' section
session_fields = root.find(".//Session/Fields")
name = find_text(session_fields, "Name")
height = find_text(session_fields, "Height")
weight = find_text(session_fields, "Weight")
plyo_day = find_text(session_fields, "Plyo_Day")
creation_date = find_text(session_fields, "Creation_date")
session_ID = find_text(session_fields, "Session_ID")

# Set default value for session_ID if it's None
if session_ID is None:
    session_ID = "1"

# Check if any required field is missing (except session_ID)
if None in [name, height, weight, plyo_day, creation_date]:
    print("One or more required fields are missing in the XML file. Exiting...")
    exit()

# Database setup
db_path = 'D:/Readiness Screen 3/Mass_Readiness_Data.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create table for the person if it doesn't exist
table_name = name.replace(' ', '_')  # Replace spaces in the name with underscores for table name
cursor.execute(f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    Height REAL,
    Weight REAL,
    Plyo_Day TEXT,
    Creation_Date TEXT,
    Session_ID TEXT
)
""")

# Insert data into the person's table
cursor.execute(f"""
INSERT INTO {table_name} (Height, Weight, Plyo_Day, Creation_Date, Session_ID)
VALUES (?, ?, ?, ?, ?)
""", (height, weight, plyo_day, creation_date, session_ID))

# Commit the changes and close the database connection
conn.commit()
conn.close()

print(f"Data successfully added to the database table '{table_name}'.")

# Spreadsheet setup
spreadsheet_path = 'D:/Readiness Screen 3/Mass Readiness Data.xlsx'
book = load_workbook(spreadsheet_path)
sheet = book.active

# Find the next empty row or insert a new row if the name already exists
start_row = 4
name_found = False
while sheet.cell(row=start_row, column=2).value is not None:
    if sheet.cell(row=start_row, column=2).value == name:
        name_found = True
        break
    start_row += 1

if name_found:
    # Insert a new row after the found row
    sheet.insert_rows(start_row + 1)
    start_row += 1
else:
    # Find the next empty row
    while sheet.cell(row=start_row, column=2).value is not None:
        start_row += 1

# Insert data into the specified cells
sheet[f'B{start_row}'] = name
sheet[f'C{start_row}'] = height
sheet[f'D{start_row}'] = weight
sheet[f'E{start_row}'] = plyo_day
sheet[f'F{start_row}'] = creation_date
sheet[f'G{start_row}'] = session_ID

# Save the updated spreadsheet
book.save(spreadsheet_path)

print("Data successfully added to the spreadsheet.")
